<a href="https://colab.research.google.com/github/chandanareddy-enugala/NLP-SLU/blob/main/NLP_Supervised_twomethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Data Loading**

In [ ]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
import random
from datetime import datetime

from sklearn.model_selection import train_test_split


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional 
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix


In [ ]:
def load_data_as_dataFrame(filePath):
  testfile = open(filePath, 'r')
  words = []
  tags = []
  for line in testfile:
      pieces = line.rstrip("\n").split("\t")
      words.append(pieces[0])
      if len(pieces)==2:
        tags.append(pieces[1])
        
  else:
    data = pd.DataFrame(columns=["words", "tags"])
    data['words'] = [str(word) for word in words]
    if len(pieces)==2:
      # --------------------------------------------------------------- Preparing DataFrame  
      data['tags'] = [str(tag) for tag in tags]

  return data

**Training Dataset**

In [ ]:
# Loading training dataset ================================
FilePath = "/content/drive/MyDrive/NLP/Assignment02/data/Input_Files/Input Files/train.tsv"
train_data = load_data_as_dataFrame(FilePath)

# WORDS ---------------------------------------------
train_words = list(train_data['words'])

# TAGS ---------------------------------------------
train_tags = list(train_data['tags'])

In [ ]:
def tag_labels(tag):
  tag_split = tag.split("-")
  tags_label = []
  for letter in tag_split:
    if len(letter)==1:
      tags_label.append("B")
    elif len(letter)==2:
      tags_label.append("B")
      tags_label.append("I")
    elif len(letter)==3:
      tags_label.append("B")
      tags_label.append("I")
      tags_label.append("I")
  return tags_label

# LABELS ---------------------------------------------
train_labels = [tag_labels(tag) for tag in train_tags]

print(f"Example :: 'k-on-s-i-l-t-an' → {tag_labels('k-on-s-i-l-t-an')}")

Example :: 'k-on-s-i-l-t-an' → ['B', 'B', 'I', 'B', 'B', 'B', 'B', 'B', 'I']


In [ ]:
train_words[0], train_tags[0], train_labels[0]

('konsiltan', 'k-on-s-i-l-t-an', ['B', 'B', 'I', 'B', 'B', 'B', 'B', 'B', 'I'])

## **Data Preprocessing**

In [ ]:
class PreprocessingData:
  def __init__(self, words, maxLength, split_str=""):
    self.data = pd.DataFrame(columns=['words'])
    self.data['words'] = words
    self.data_params = {"maxLength":maxLength, "split_str":split_str}
    self.final_array = np.array([])

  def words_split(self, words="None", split_str="None"):
    if words=="None":
      words = list(self.data['words'])
    if split_str=="None":
      split_str = self.data_params['split_str']

    if split_str=="":
      words_split = [[letter.lower() for letter in list(word)] for word in words]
    else:
      words_split =  [[letter.lower() for letter in word.split("-")] for word in words]
    self.data['words_split'] = words_split
    return words_split

  def find_UniqueLetters(self, words_split="None"):
    if words_split=="None":
      words_split = list(self.data['words_split'])

    UniqueLetters = ["<PAD>"]+list({letter for word in words_split for letter in word})
    UniqueLettersLength = len(UniqueLetters)
    Letters2Num = {w:i for i, w in enumerate(UniqueLetters)}
    Num2Letters = {i:w for w, i in Letters2Num.items()}
    
    self.data_params['UniqueLetters'] = UniqueLetters
    self.data_params['UniqueLettersLength'] = UniqueLettersLength
    self.data_params['Letters2Num'] = Letters2Num
    self.data_params['Num2Letters'] = Num2Letters
    self.data_params['denseValue'] = UniqueLettersLength
    return UniqueLetters, UniqueLettersLength, Letters2Num, Num2Letters

  def convert_Text2Num(self, words_split="None", Letters2Num="None"):
    if words_split=="None":
      words_split = list(self.data['words_split'])
    if Letters2Num == "None":
      Letters2Num = self.data_params['Letters2Num']
    elif "Letters2Num" not in self.data_params.keys():
      self.data_params['Letters2Num']=Letters2Num
      self.data_params['Num2Letters']= {i:w for w, i in Letters2Num.items()}

    words_split_num = [[Letters2Num[letter] for letter in word] for word in words_split]
    self.data['words_split_num'] = words_split_num
    return words_split_num

  def padding(self, words_split_num="None", maxLength="None", Letters2Num="None"):
    if words_split_num == "None":
      words_split_num = list(self.data['words_split_num'])
    if maxLength == "None":
      maxLength = self.data_params['maxLength']
    if Letters2Num =="None":
      Letters2Num = self.data_params['Letters2Num']

    words_split_num_pad = [word+[Letters2Num["<PAD>"] for i in range(maxLength-len(word))] for word in words_split_num]
    self.data['words_split_num_pad'] = words_split_num_pad
    return words_split_num_pad
    
  def getData_into_Array(self, words_split_num_pad="None", denseValue="None"):
    if words_split_num_pad == "None":
      words_split_num_pad = np.array(data['words_split_num_pad'].tolist())
    if denseValue == "None":
      denseValue = self.data_params['denseValue']
    if denseValue != "None":
      self.final_array = to_categorical(words_split_num_pad, num_classes=denseValue)
    return self.final_array

**Words**

In [ ]:
words_DP = PreprocessingData(train_words, 20)
_ = words_DP.words_split()
_ = words_DP.find_UniqueLetters()
# words_DP.data_params
_ = words_DP.convert_Text2Num()
_ = words_DP.padding()

words_array = np.array(words_DP.data['words_split_num_pad'].tolist())
# words_array = words_DP.getData_into_Array()

print("words_array shape : ", words_array.shape)

words_array shape :  (12812, 20)


In [ ]:
words_DP.data.head()

,words,words_split,words_split_num,words_split_num_pad
0,konsiltan,"[k, o, n, s, i, l, t, a, n]","[14, 9, 22, 6, 24, 15, 16, 3, 22]","[14, 9, 22, 6, 24, 15, 16, 3, 22, 0, 0, 0, 0, ..."
1,depotwa,"[d, e, p, o, t, w, a]","[21, 11, 10, 9, 16, 1, 3]","[21, 11, 10, 9, 16, 1, 3, 0, 0, 0, 0, 0, 0, 0,..."
2,sosyopwofesyonèl,"[s, o, s, y, o, p, w, o, f, e, s, y, o, n, è, l]","[6, 9, 6, 19, 9, 10, 1, 9, 13, 11, 6, 19, 9, 2...","[6, 9, 6, 19, 9, 10, 1, 9, 13, 11, 6, 19, 9, 2..."
3,vejetal,"[v, e, j, e, t, a, l]","[8, 11, 12, 11, 16, 3, 15]","[8, 11, 12, 11, 16, 3, 15, 0, 0, 0, 0, 0, 0, 0..."
4,repibliye,"[r, e, p, i, b, l, i, y, e]","[26, 11, 10, 24, 5, 15, 24, 19, 11]","[26, 11, 10, 24, 5, 15, 24, 19, 11, 0, 0, 0, 0..."


In [ ]:
[words_DP.data_params['Num2Letters'][i] for i in words_array[0] if i!=0]

['k', 'o', 'n', 's', 'i', 'l', 't', 'a', 'n']

**Tags**

In [ ]:
tags_DP = PreprocessingData(train_tags, 20, "-")
_ = tags_DP.words_split()
_ = tags_DP.find_UniqueLetters()
# tags_DP.data_params
_ = tags_DP.convert_Text2Num()
_ = tags_DP.padding()

tags_split_num_pad = np.array(tags_DP.data["words_split_num_pad"].tolist())
tags_array = to_categorical(tags_split_num_pad, num_classes=tags_DP.data_params["UniqueLettersLength"])

# tags_array = tags_DP.getData_into_Array()
print("tags_array shape : ", tags_array.shape)

tags_array shape :  (12812, 20, 33)


In [ ]:
tags_DP.data_params.keys()

dict_keys(['maxLength', 'split_str', 'UniqueLetters', 'UniqueLettersLength', 'Letters2Num', 'Num2Letters', 'denseValue'])

In [ ]:
tags_DP.data.head()

,words,words_split,words_split_num,words_split_num_pad
0,k-on-s-i-l-t-an,"[k, on, s, i, l, t, an]","[20, 7, 10, 29, 21, 22, 5]","[20, 7, 10, 29, 21, 22, 5, 0, 0, 0, 0, 0, 0, 0..."
1,d-e-p-o-t-w-a,"[d, e, p, o, t, w, a]","[27, 16, 15, 14, 22, 1, 3]","[27, 16, 15, 14, 22, 1, 3, 0, 0, 0, 0, 0, 0, 0..."
2,s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l,"[s, o, s, y, o, p, w, o, f, e, s, y, o, n, è, l]","[10, 14, 10, 25, 14, 15, 1, 14, 19, 16, 10, 25...","[10, 14, 10, 25, 14, 15, 1, 14, 19, 16, 10, 25..."
3,v-e-j-e-t-a-l,"[v, e, j, e, t, a, l]","[13, 16, 17, 16, 22, 3, 21]","[13, 16, 17, 16, 22, 3, 21, 0, 0, 0, 0, 0, 0, ..."
4,r-e-p-i-b-l-i-y-e,"[r, e, p, i, b, l, i, y, e]","[31, 16, 15, 29, 8, 21, 29, 25, 16]","[31, 16, 15, 29, 8, 21, 29, 25, 16, 0, 0, 0, 0..."


In [ ]:
tags_array1 = np.argmax(tags_array, axis=-1)
[tags_DP.data_params['Num2Letters'][i] for i in tags_array1[0] if i!=0]

['k', 'on', 's', 'i', 'l', 't', 'an']

**Labels**

In [ ]:
labels_DP = PreprocessingData(train_labels, 20, "")
labels_DP.data['words_split'] = train_labels
_ = labels_DP.find_UniqueLetters()
_ = labels_DP.convert_Text2Num()
_ = labels_DP.padding()

labels_split_num_pad = np.array(labels_DP.data["words_split_num_pad"].tolist())
labels_array = to_categorical(labels_split_num_pad, num_classes=3)

print("labels_array shape : ", labels_array.shape)

labels_array shape :  (12812, 20, 3)


In [ ]:
labels_DP.data.head()

,words,words_split,words_split_num,words_split_num_pad
0,"[B, B, I, B, B, B, B, B, I]","[B, B, I, B, B, B, B, B, I]","[1, 1, 2, 1, 1, 1, 1, 1, 2]","[1, 1, 2, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, ..."
1,"[B, B, B, B, B, B, B]","[B, B, B, B, B, B, B]","[1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[B, B, B, B, B, B, B, B, B, B, B, B, B, B, B, B]","[B, B, B, B, B, B, B, B, B, B, B, B, B, B, B, B]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[B, B, B, B, B, B, B]","[B, B, B, B, B, B, B]","[1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[B, B, B, B, B, B, B, B, B]","[B, B, B, B, B, B, B, B, B]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
labels_array1 = np.argmax(labels_array, axis=-1)
[labels_DP.data_params['Num2Letters'][i] for i in labels_array1[0] if i!=0]

['B', 'B', 'I', 'B', 'B', 'B', 'B', 'B', 'I']

## **Model Preparation & Training**

In [ ]:
class HmmModel:
  def __init__(self, X, Y, input_dim, input_length, dense_val, Num2Letters):
    self.X = X
    self.Y = Y
    self.input_dim = input_dim
    self.input_length = input_length
    self.dense_val = dense_val
    self.Num2Letters = Num2Letters
    self.Y_pred = []
    self.Y_pred_text = []

    self.model = Sequential()
    self.model.add(Embedding(input_dim=self.input_dim, output_dim=50, input_length=self.input_length))
    self.model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.2)))
    self.model.add(TimeDistributed(Dense(self.dense_val, activation="softmax")))
    self.model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Summary %%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print(self.model.summary())
    print("\n")
  
  def train(self):
    startTime = datetime.now()
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Training %%%%%%%%%%%%%%%%%%%%%%%%%%%")
    history = self.model.fit(self.X, self.Y, batch_size=5, epochs=1, validation_split=0.2, verbose=1)
    endTime = datetime.now()
    print(f"Training is completed Successfully in Time : {endTime-startTime}")
    print("\n")
  
  def predict_result(self, X):
    startTime = datetime.now()
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Predictions %%%%%%%%%%%%%%%%%%%%%%%%%%%")
    self.Y_pred = self.model.predict(X)
    self.Y_pred = np.argmax(self.Y_pred, axis=-1)
    endTime = datetime.now()
    print(f"Prediction is completed Successfully in Time : {endTime-startTime}")
    print("\n")
    return self.Y_pred
  
  def predict_text(self, Y_pred="None"):
    startTime = datetime.now()
    if Y_pred == "None":
      Y_pred = self.Y_pred

    if len(Y_pred)==0:
      print("----------------------- WARNING -----------------------")
      print("Caution: Run function : HmmModel.predict(input)")
      print("\n")
      return ""
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Prediction Strings %%%%%%%%%%%%%%%%%%%%%%%%%%%")
    for tag_num in Y_pred:
      tag = ""
      for j in [n for n in tag_num if n!=0]:
        tag += self.Num2Letters[j] +"-"
      else:
        tag = tag[:-1]
      self.Y_pred_text.append(tag)
    endTime = datetime.now()
    print(f"Prediction is completed Successfully in Time : {endTime-startTime}")
    print("\n")
    return self.Y_pred_text
  
  def evaluate_ModelAccuracy(self, X, Y):
    Y_actual = np.argmax(Y, axis=-1)
    Y_pred = self.predict_result(X)
    Y_actual = [n for i in range(len(Y_actual)) for n in Y_actual[i]]
    Y_pred = [n for i in range(len(Y_pred)) for n in Y_pred[i]]
    
    total_length = len(Y_actual)
    count = 0
    for i in range(total_length):
      if Y_actual[i]==Y_pred[i]:
        count += 1
    print(f"Model accuracy : {np.round((count/total_length)*100, 2)}%", )
    return ""
  
  def saveModel(self, FilePath):
    self.model.save(FilePath)
    print("Model is saved successfully")
    return ""

  def loadModel(self, FilPath):
    self.model = self.model.load_weights(FilePath)
    print("Model is loaded successfully")
    return ""

### **Model training: Words to Tags**

In [ ]:
X = words_array
Y = tags_array
input_dim = words_DP.data_params['UniqueLettersLength'] # words_A_UniqueLettersLength # 28
input_length = words_DP.data_params['maxLength'] # words_A_MaxLength # 20
dense_val = Y.shape[2] # 33
Num2Letters = tags_DP.data_params['Num2Letters']


print(f"Input Dimension : {input_dim}")
print(f"Input Length : {input_length}")
print(f"Dense : {dense_val}")


model_Words2Tags = HmmModel(X, Y, input_dim, input_length, dense_val, Num2Letters)
model_Words2Tags.train()
_ = model_Words2Tags.evaluate_ModelAccuracy(X, Y)

train_tags_pred = model_Words2Tags.predict_result(X)
train_tags_pred_text = model_Words2Tags.predict_text()

Input Dimension : 28
Input Length : 20
Dense : 33
%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Summary %%%%%%%%%%%%%%%%%%%%%%%%%%%
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 50)            1400      
                                                                 
 bidirectional (Bidirectiona  (None, 20, 200)          120800    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 20, 33)           6633      
 ibuted)                                                         
                                                                 
Total params: 128,833
Trainable params: 128,833
Non-trainable params: 0
_________________________________________________________________
None


%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Training %%%%%%%%%%%%

In [ ]:
train_result_df = pd.DataFrame()
train_result_df['actual_words'] = words_DP.data['words']
train_result_df['actual_tags'] = list(tags_DP.data['words'])
train_result_df['predicted_tags'] = train_tags_pred_text
train_result_df.head()

,actual_words,actual_tags,predicted_tags
0,konsiltan,k-on-s-i-l-t-an,k-on-s-i-l-t-an
1,depotwa,d-e-p-o-t-w-a,d-e-p-o-t-w-a
2,sosyopwofesyonèl,s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l,s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l
3,vejetal,v-e-j-e-t-a-l,v-e-j-e-t-a-l
4,repibliye,r-e-p-i-b-l-i-y-e,r-e-p-i-b-l-i-y-e


In [ ]:
FilePath = "/content/drive/MyDrive/NLP/Assignment02/data/Output_Files/train_result_from_model_W2T.txt"
train_file = open(FilePath,"w")

for i in range(len(train_result_df)):
  text = str(train_result_df.loc[i, "actual_words"])+"\t"+str(train_result_df.loc[i, "predicted_tags"])+"\n"
  train_file.write(text)
train_file.close()

In [ ]:
train_result_df.to_csv("/content/drive/MyDrive/NLP/Assignment02/data/Output_Files/train_result_from_model_W2T.csv", index=False)

In [ ]:
model_Words2Tags.saveModel("/content/drive/MyDrive/NLP/Assignment02/data/Models/model_Words2Tags.h5")

Model is saved successfully


''

### **Model training: Words to Labels**

In [ ]:
X = words_array
Y = labels_array
input_dim = words_DP.data_params['UniqueLettersLength'] # words_A_UniqueLettersLength # 28
input_length = words_DP.data_params['maxLength'] # words_A_MaxLength # 20
dense_val = Y.shape[2] # 3
labels_Num2Letters = labels_DP.data_params['Num2Letters']


print(f"Input Dimension : {input_dim}")
print(f"Input Length : {input_length}")
print(f"Dense : {dense_val}")


model_Words2Labels = HmmModel(X, Y, input_dim, input_length, dense_val, labels_Num2Letters)
model_Words2Labels.train()
_ = model_Words2Labels.evaluate_ModelAccuracy(X, Y)

Input Dimension : 28
Input Length : 20
Dense : 3
%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Summary %%%%%%%%%%%%%%%%%%%%%%%%%%%
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 50)            1400      
                                                                 
 bidirectional_1 (Bidirectio  (None, 20, 200)          120800    
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, 20, 3)            603       
 tributed)                                                       
                                                                 
Total params: 122,803
Trainable params: 122,803
Non-trainable params: 0
_________________________________________________________________
None


%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Training %%%%%%%%%%%

In [ ]:
train_labels_pred = model_Words2Labels.predict_result(X)
train_labels_pred_text = model_Words2Labels.predict_text()

%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Predictions %%%%%%%%%%%%%%%%%%%%%%%%%%%
Prediction is completed Successfully in Time : 0:00:04.347110


%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Prediction Strings %%%%%%%%%%%%%%%%%%%%%%%%%%%
Prediction is completed Successfully in Time : 0:00:00.115286




In [ ]:
train_result_df2 = pd.DataFrame()
train_result_df2['actual_words'] = words_DP.data['words']
train_result_df2['actual_labels'] = list(labels_DP.data['words'])
train_result_df2['predicted_labels'] = [word.split("-") for word in train_labels_pred_text]
train_result_df2.head()

,actual_words,actual_labels,predicted_labels
0,konsiltan,"[B, B, I, B, B, B, B, B, I]","[B, B, I, B, B, B, B, B, I]"
1,depotwa,"[B, B, B, B, B, B, B]","[B, B, B, B, B, B, B]"
2,sosyopwofesyonèl,"[B, B, B, B, B, B, B, B, B, B, B, B, B, B, B, B]","[B, B, B, B, B, B, B, B, B, B, B, B, B, B, B, B]"
3,vejetal,"[B, B, B, B, B, B, B]","[B, B, B, B, B, B, B]"
4,repibliye,"[B, B, B, B, B, B, B, B, B]","[B, B, B, B, B, B, B, B, B]"


In [ ]:
def find_Labels2Num(label):
  label_rev_val = []
  label_rev = list(reversed(label))
  label_val = []
  for val in label_rev:
    if val=="I":
      label_val.append(val)
    elif val=="B":
      label_val.append(val)
      label_rev_val.append(len(label_val))
      label_val = []
  label_val = list(reversed(label_rev_val))
  return label_val
train_result_df2['predicted_labels_num'] = train_result_df2['predicted_labels'].apply(find_Labels2Num) 

In [ ]:
def find_tags(word, label_num):
  tag = ""
  for j in label_num:
    tag += word[0:j]+"-"
    word = word[j:]
  return tag

train_result_df2["predicted_tags"] = train_result_df2[["actual_words", 'predicted_labels_num']].apply(lambda x: find_tags(x["actual_words"], x['predicted_labels_num']), axis=1)

In [ ]:
train_result_df2.head()

,actual_words,actual_labels,predicted_labels,predicted_labels_num,predicted_tags
0,konsiltan,"[B, B, I, B, B, B, B, B, I]","[B, B, I, B, B, B, B, B, I]","[1, 2, 1, 1, 1, 1, 2]",k-on-s-i-l-t-an-
1,depotwa,"[B, B, B, B, B, B, B]","[B, B, B, B, B, B, B]","[1, 1, 1, 1, 1, 1, 1]",d-e-p-o-t-w-a-
2,sosyopwofesyonèl,"[B, B, B, B, B, B, B, B, B, B, B, B, B, B, B, B]","[B, B, B, B, B, B, B, B, B, B, B, B, B, B, B, B]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l-
3,vejetal,"[B, B, B, B, B, B, B]","[B, B, B, B, B, B, B]","[1, 1, 1, 1, 1, 1, 1]",v-e-j-e-t-a-l-
4,repibliye,"[B, B, B, B, B, B, B, B, B]","[B, B, B, B, B, B, B, B, B]","[1, 1, 1, 1, 1, 1, 1, 1, 1]",r-e-p-i-b-l-i-y-e-


In [ ]:
FilePath = "/content/drive/MyDrive/NLP/Assignment02/data/Output_Files/train_result_from_model_W2L2T.txt"
train_file = open(FilePath,"w")

for i in range(len(train_result_df2)):
  text = str(train_result_df2.loc[i, "actual_words"])+"\t"+str(train_result_df2.loc[i, "predicted_tags"])+"\n"
  train_file.write(text)
train_file.close()

In [ ]:
train_result_df2.to_csv("/content/drive/MyDrive/NLP/Assignment02/data/Output_Files/train_result_from_model_WL22T.csv", index=False)

In [ ]:
model_Words2Labels.saveModel("/content/drive/MyDrive/NLP/Assignment02/data/Models/model_Words2Labels.h5")

Model is saved successfully


''

## **Predictions**

**Load Test dataset**

In [ ]:
# Loading training dataset ================================
FilePath = "/content/drive/MyDrive/NLP/Assignment02/data/Input_Files/test.txt"
test_data = load_data_as_dataFrame(FilePath)

# WORDS ---------------------------------------------
test_words = list(test_data['words'])

**Preprocessing**

In [ ]:
test_words_DP = PreprocessingData(test_words, 20)
_ = test_words_DP.words_split()
_ = test_words_DP.convert_Text2Num(Letters2Num = words_DP.data_params['Letters2Num'])
_ = test_words_DP.padding()

test_words_array = np.array(test_words_DP.data['words_split_num_pad'].tolist())
print(test_words_array.shape)

(1427, 20)


### Using **Model : Words to Tags**

In [ ]:
test_tags_pred = model_Words2Tags.predict_result(test_words_array)

%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Predictions %%%%%%%%%%%%%%%%%%%%%%%%%%%
Prediction is completed Successfully in Time : 0:00:00.675053




In [ ]:
startTime = datetime.now()
tag_Num2Letters = tags_DP.data_params['Num2Letters']
Y_test_pred_text = []
for tag_num in test_tags_pred:
  tag = ""
  for j in [n for n in tag_num if n!=0]:
    tag += tag_Num2Letters[j] +"-"
    
  else:
    tag = tag[:-1]
  Y_test_pred_text.append(tag)
endTime = datetime.now()
print(f"Prediction is completed Successfully in Time : {endTime-startTime}")

Prediction is completed Successfully in Time : 0:00:00.017525


In [ ]:
len(words_DP.data['words']), len(Y_test_pred_text)

(12812, 1427)

In [ ]:
test_result_df1 = pd.DataFrame()
test_result_df1['actual_words'] = test_words_DP.data['words']
test_result_df1['predicted_tags'] = list(Y_test_pred_text)
test_result_df1.head()

,actual_words,predicted_tags
0,Oradye,o-r-a-d-y-e
1,bous,b-ou-s
2,titan,t-i-t-an
3,refi,r-e-f-i
4,netwayè,n-e-t-w-y-è-è


In [ ]:
FilePath = "/content/drive/MyDrive/NLP/Assignment02/data/Output_Files/test_result_from_model_W2T.txt"
train_file = open(FilePath,"w")

for i in range(len(test_result_df1)):
  text = str(test_result_df1.loc[i, "actual_words"])+"\t"+str(test_result_df1.loc[i, "predicted_tags"])+"\n"
  train_file.write(text)
train_file.close()

In [ ]:
test_result_df1.to_csv("/content/drive/MyDrive/NLP/Assignment02/data/Output_Files/test_result_from_model_W2T.csv", index=False)

### Using **Model : Words to Label to Tags**

In [ ]:
test_labels_pred = model_Words2Labels.predict_result(test_words_array)
print("test_labels_pred shape : ", test_labels_pred.shape)

%%%%%%%%%%%%%%%%%%%%%%%%%%% Model Predictions %%%%%%%%%%%%%%%%%%%%%%%%%%%
Prediction is completed Successfully in Time : 0:00:00.531268


test_labels_pred shape :  (1427, 20)


In [ ]:
def find_Labels2Num(label):
  label_rev_val = []
  label_rev = list(reversed(label))
  label_val = []
  for val in label_rev:
    if val=="I":
      label_val.append(val)
    elif val=="B":
      label_val.append(val)
      label_rev_val.append(len(label_val))
      label_val = []
  label_val = list(reversed(label_rev_val))
  return label_val

def find_tags(word, label_num):
  tag = ""
  for j in label_num:
    tag += word[0:j]+"-"
    word = word[j:]
  return tag[:-1]

In [ ]:
startTime = datetime.now()
tag_Num2Letters = labels_DP.data_params['Num2Letters']
Y_pred_text = []
for tag_num in test_labels_pred:
  tag = ""
  for j in [n for n in tag_num if n!=0]:
    tag += tag_Num2Letters[j] +"-"
  else:
    tag = tag[:-1]
  Y_pred_text.append(tag)
endTime = datetime.now()
print(f"Prediction is completed Successfully in Time : {endTime-startTime}")


test_result_df2 = pd.DataFrame()
test_result_df2['actual_words'] = test_words_DP.data['words']
test_result_df2['predicted_labels'] = [word.split("-") for word in Y_pred_text]

test_result_df2['predicted_labels_num'] = test_result_df2['predicted_labels'].apply(find_Labels2Num) 
test_result_df2["predicted_tags"] = test_result_df2[["actual_words", 'predicted_labels_num']].apply(lambda x: find_tags(x["actual_words"], x['predicted_labels_num']), axis=1)

test_result_df2.head()

Prediction is completed Successfully in Time : 0:00:00.014778


,actual_words,predicted_labels,predicted_labels_num,predicted_tags
0,Oradye,"[B, B, B, B, B, B]","[1, 1, 1, 1, 1, 1]",O-r-a-d-y-e
1,bous,"[B, B, I, B]","[1, 2, 1]",b-ou-s
2,titan,"[B, B, B, B, I]","[1, 1, 1, 2]",t-i-t-an
3,refi,"[B, B, B, B]","[1, 1, 1, 1]",r-e-f-i
4,netwayè,"[B, B, B, B, B, B, B]","[1, 1, 1, 1, 1, 1, 1]",n-e-t-w-a-y-è


In [ ]:
FilePath = "/content/drive/MyDrive/NLP/Assignment02/data/Output_Files/test_result_from_model_W2L2T.txt"
train_file = open(FilePath,"w")

for i in range(len(test_result_df2)):
  text = str(test_result_df2.loc[i, "actual_words"])+"\t"+str(test_result_df2.loc[i, "predicted_tags"])+"\n"
  train_file.write(text)
train_file.close()

In [ ]:
test_result_df2.to_csv("/content/drive/MyDrive/NLP/Assignment02/data/Output_Files/test_result_from_model_W2L2T.csv", index=False)